<a href="https://colab.research.google.com/github/MDankloff/ClusterCompas/blob/main/BAFV2data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [74]:
import pickle
import lightgbm as lgbm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import glob
import random
import os
import joblib
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, roc_auc_score
from sklearn.utils import resample

In [75]:
!pip install dask[dataframe]

In [76]:
from google.colab import drive
drive.mount('/content/drive')
! cd '/content/drive/MyDrive/Mirthe_Supervision /Paper#3/BAF'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# LOAD DATA & MODEL

In [77]:
base_path = '/content/drive/MyDrive/Mirthe_Supervision /Paper#3/BAF/parquet data'

extension = "parquet" #for smaller "parquet" depending on the downloaded file
data_paths = glob.glob(f"{base_path}/*.{extension}")

def read_dataset(path, ext = extension):
    if ext == "csv":
      return pd.read_csv(path)
    elif ext == "parquet":
      return pd.read_parquet(path)
    else:
      raise ValueError(f"Unsupported file extension: {ext}")

# Extract variant name from the file path (without the extension)
def get_variant(path):
    return os.path.basename(path).split(".")[0]

# Dictionary comprehension to read all CSV files into a dictionary of DataFrames
dataframes = {
    get_variant(path): read_dataset(path) for path in data_paths
}
print(f"Loaded datasets: {list(dataframes.keys())}")

datasets_paths = {
    "Base": base_path + "/Base.parquet", # sampled to best represent original dataset
    "Variant I": base_path + "/Variant I.parquet", # higher group size disparity than base - reducing the size of the minority group from approx 20 - 10% of the dataset
    "Variant II": base_path + "/Variant II.parquet", # higher prevalence disparity than base - one group has 5 x the fraud detection rate of the other while group sizes are equal
    "Variant III": base_path + "/Variant III.parquet", # better separability for one of the groups -
    "Variant IV": base_path + "/Variant IV.parquet", # higher prevalence disparity in train
    "Variant V": base_path + "/Variant V.parquet", # better separability in train for one of the groups
}


Loaded datasets: ['Base', 'Variant I', 'Variant II', 'Variant III', 'Variant IV', 'Variant V']


load best light gbm model from variant 2

In [78]:
'''# directory containing the model files
model_dir = '/content/drive/MyDrive/Mirthe_Supervision /Paper#3/BAF/Best Model per Variant/Accuracy'

# Get list of all model files in the directory
model_files = glob.glob(os.path.join(model_dir, '*.pkl'))

# Dictionary to store loaded models
models = {}

# Load all models from the directory and save them to the dictionary
for model_file in model_files:
    # Load the model
    with open(model_file, 'rb') as f:
        model = joblib.load(f)

    # Extract the model name from the file path (without extension)
    model_name = os.path.basename(model_file).split('.')[0]

    # Add the model to the dictionary
    models[model_name] = model

    # Optional: Save the model back (though it seems redundant here)
    save_path = os.path.join(model_dir, f'{model_name}.pkl')
    joblib.dump(model, save_path)

    print(f"Model '{model_name}' loaded and saved to: {save_path}")

# Accessing the best model for variant 2
modelv2 = models.get("model_Variant II_top_4")

print(modelv2)'''

'# directory containing the model files\nmodel_dir = \'/content/drive/MyDrive/Mirthe_Supervision /Paper#3/BAF/Best Model per Variant/Accuracy\'\n\n# Get list of all model files in the directory\nmodel_files = glob.glob(os.path.join(model_dir, \'*.pkl\'))\n\n# Dictionary to store loaded models\nmodels = {}\n\n# Load all models from the directory and save them to the dictionary\nfor model_file in model_files:\n    # Load the model\n    with open(model_file, \'rb\') as f:\n        model = joblib.load(f)\n\n    # Extract the model name from the file path (without extension)\n    model_name = os.path.basename(model_file).split(\'.\')[0]\n\n    # Add the model to the dictionary\n    models[model_name] = model\n\n    # Optional: Save the model back (though it seems redundant here)\n    save_path = os.path.join(model_dir, f\'{model_name}.pkl\')\n    joblib.dump(model, save_path)\n\n    print(f"Model \'{model_name}\' loaded and saved to: {save_path}")\n\n# Accessing the best model for variant 2

# Explore data

In [79]:
#rename dataset variant 2 and create a copy
v2 = dataframes["Variant II"]
v2_old = v2.copy()

In [80]:
pd.set_option('display.max_columns', 50) # Increase the maximum number of columns displayed in Pandas to 50
pd.set_option('display.max_rows', 50)
v2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 32 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   fraud_bool                        1000000 non-null  int64  
 1   income                            1000000 non-null  float64
 2   name_email_similarity             1000000 non-null  float64
 3   prev_address_months_count         1000000 non-null  int64  
 4   current_address_months_count      1000000 non-null  int64  
 5   customer_age                      1000000 non-null  int64  
 6   days_since_request                1000000 non-null  float64
 7   intended_balcon_amount            1000000 non-null  float64
 8   payment_type                      1000000 non-null  object 
 9   zip_count_4w                      1000000 non-null  int64  
 10  velocity_6h                       1000000 non-null  float64
 11  velocity_24h                      1000

In [81]:
v2['payment_type'].unique() #remove because anonymized
v2['days_since_request'].unique() #remove because unclear values

array([0.01673985, 0.01900183, 0.04706417, ..., 0.03555377, 0.02066269,
       0.0074784 ])

In [82]:
print(v2['days_since_request'])

0         0.016740
1         0.019002
2         0.047064
3         0.008007
4         2.513544
            ...   
999995    0.001835
999996    0.023952
999997    0.035554
999998    0.020663
999999    0.007478
Name: days_since_request, Length: 1000000, dtype: float64


In [83]:
v2_old['source'].unique()
v2_old['device_os'].unique()
v2_old['keep_alive_session'].unique()
#v2_old['device_fraud_count'].unique()

array([0, 1])

In [84]:
1 in v2_old['device_fraud_count'].unique()

False

how to deal with missing values

In [97]:
v2_old['device_os'].unique()

array(['other', 'macintosh', 'linux', 'windows', 'x11'], dtype=object)

Master dataset

In [98]:
#FEATURES
META = ['clusters', 'new_clusters']
ERROR = ['errors', 'TP', 'TN', 'FN', 'FP']
REG = ['bank_branch_count_8w', 'credit_risk_score', 'device_os','month', 'session_length_in_minutes', 'email_is_free', 'proposed_credit_limit', 'name_email_similarity',
      'zip_count_4w', 'date_of_birth_distinct_emails_4w', 'phone_mobile_valid', 'has_other_cards', 'foreign_request']
SEN = ['customer_age', 'income'] #protected attributes in BAF paper
DUMMY = ['source_INTERNET', 'Source_TELEAPP', 'device_os_other', 'device_os_macintosh','device_os_linux','device_os_windows', 'device_os_x11']

#FEATURES SCALED
ERROR_scaled = ['errors_scaled']
REG_scaled = ['bank_branch_count_8w_scaled', 'credit_risk_score_scaled', 'device_os_scaled','month_scaled', 'session_length_in_minutes_scaled',
              'email_is_free_scaled', 'proposed_credit_limit_scaled', 'name_email_similarity_scaled', 'zip_count_4w_scaled',
              'date_of_birth_distinct_emails_4w_scaled', 'phone_home_valid_scaled','has_other_cards_scaled', 'foreign_request_scaled']
SEN_scaled = ['customer_age_scaled', 'income_scaled']
DUMMY_scaled = ['source_INTERNET_scaled', 'Source_TELEAPP_scaled', 'device_os_other_scaled', 'device_os_macintosh_scaled', 'device_os_linux_scaled', 'device_os_windows_scaled', 'device_os_x11_scaled']


#SHAP FEATURES
SHAP_REG = ['bank_branch_count_8w_shap', 'credit_risk_score_shap', 'device_os_shap', 'month_shap', 'session_length_in_minutes_shap', 'email_is_free_shap', 'proposed_credit_limit_shap', 'name_email_similarity_shap',
 'zip_count_4w_shap', 'date_of_birth_distinct_emails_4w_shap', 'phone_mobile_valid_shap', 'has_other_cards_shap', 'foreign_request_shap']
SHAP_SEN = ['customer_age_shap', 'income_shap']
SHAP_DUMMY = ['source_INTERNET_shap', 'Source_TELEAPP_shap', 'device_os_other_shap', 'device_os_macintosh_shap','device_os_linux_shap','device_os_windows_shap', 'device_os_x11_shap']

#SHAP FEATURES SCALED
SHAP_REG_scaled = ['bank_branch_count_8w_shap_scaled', 'credit_risk_score_shap_scaled', 'device_os_shap_scaled', 'month_shap_scaled', 'session_length_in_minutes_shap_scaled',
 'email_is_free_shap_scaled', 'proposed_credit_limit_shap_scaled', 'name_email_similarity_shap_scaled', 'zip_count_4w_shap_scaled',
 'date_of_birth_distinct_emails_4w_shap_scaled', 'phone_mobile_valid_shap_scaled', 'has_other_cards_shap_scaled', 'foreign_request_shap_scaled']
SHAP_SEN_scaled = ['customer_age_shap_scaled', 'income_shap_scaled']
SHAP_DUMMY_scaled = ['source_INTERNET_shap_scaled', 'Source_TELEAPP_shap_scaled', 'device_os_other_shap_scaled', 'device_os_macintosh_shap_scaled',
 'device_os_linux_shap_scaled', 'device_os_windows_shap_scaled', 'device_os_x11_shap_scaled']

'''removed features = 'device_fraud_count', 'intended_balcon_amount', 'payment_type', 'days_since_request',
'velocity_6h', 'velocity_24h', 'velocity_4w', 'keep_alive_session', 'prev_address_months_count', 'current_address_months_count', 'phone_home_valid', 'bank_months_count', 'device_distinct_emails_8w', 'housing_status', 'employment_status' '''

"removed features = 'device_fraud_count', 'intended_balcon_amount', 'payment_type', 'days_since_request', \n'velocity_6h', 'velocity_24h', 'velocity_4w', 'keep_alive_session', 'prev_address_months_count', 'current_address_months_count', 'phone_home_valid', 'bank_months_count', 'device_distinct_emails_8w', 'housing_status', 'employment_status' "

In [86]:
v2_old['housing_status'].unique()

array(['BC', 'BA', 'BE', 'BB', 'BD', 'BF', 'BG'], dtype=object)

In [87]:
v2_old['employment_status'].unique()

array(['CB', 'CA', 'CE', 'CD', 'CC', 'CF', 'CG'], dtype=object)

In [88]:
v2_old['source'].unique()

array(['INTERNET', 'TELEAPP'], dtype=object)

In [95]:
v2_old['device_os'].unique()

array(['other', 'macintosh', 'linux', 'windows', 'x11'], dtype=object)

Remove features

In [89]:
features_to_remove = ['device_fraud_count', 'intended_balcon_amount', 'payment_type', 'days_since_request',
'velocity_6h', 'velocity_24h', 'velocity_4w', 'keep_alive_session', 'prev_address_months_count', 'current_address_months_count', 'phone_home_valid', 'bank_months_count', 'device_distinct_emails_8w', 'housing_status', 'employment_status' ]

v2 = v2.drop(columns = features_to_remove)

In [90]:
v2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 17 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   fraud_bool                        1000000 non-null  int64  
 1   income                            1000000 non-null  float64
 2   name_email_similarity             1000000 non-null  float64
 3   customer_age                      1000000 non-null  int64  
 4   zip_count_4w                      1000000 non-null  int64  
 5   bank_branch_count_8w              1000000 non-null  int64  
 6   date_of_birth_distinct_emails_4w  1000000 non-null  int64  
 7   credit_risk_score                 1000000 non-null  int64  
 8   email_is_free                     1000000 non-null  int64  
 9   phone_mobile_valid                1000000 non-null  int64  
 10  has_other_cards                   1000000 non-null  int64  
 11  proposed_credit_limit             1000

In [91]:
#get a smaller sample
v2_sample = v2.sample(n= 8000, random_state = 42)

In [92]:
v2_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8000 entries, 987231 to 534950
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   fraud_bool                        8000 non-null   int64  
 1   income                            8000 non-null   float64
 2   name_email_similarity             8000 non-null   float64
 3   customer_age                      8000 non-null   int64  
 4   zip_count_4w                      8000 non-null   int64  
 5   bank_branch_count_8w              8000 non-null   int64  
 6   date_of_birth_distinct_emails_4w  8000 non-null   int64  
 7   credit_risk_score                 8000 non-null   int64  
 8   email_is_free                     8000 non-null   int64  
 9   phone_mobile_valid                8000 non-null   int64  
 10  has_other_cards                   8000 non-null   int64  
 11  proposed_credit_limit             8000 non-null   float64
 12  fore

In [93]:
'''missing_values = v2_sample.isna()
print(missing_values)'''

'missing_values = v2_sample.isna()\nprint(missing_values)'

In [94]:
#print(v2_sample.loc[534950])

fraud_bool                                  0
income                                    0.6
name_email_similarity                0.885378
customer_age                               40
zip_count_4w                             1870
bank_branch_count_8w                        6
date_of_birth_distinct_emails_4w           11
credit_risk_score                          99
email_is_free                               1
phone_mobile_valid                          1
has_other_cards                             1
proposed_credit_limit                   200.0
foreign_request                             0
source                               INTERNET
session_length_in_minutes           13.034532
device_os                               linux
month                                       3
Name: 534950, dtype: object


Get dummies for objects

In [96]:
dummy_cols = ['source', 'device_os']
v2_sample = pd.get_dummies(v2_sample, columns = dummy_cols)
v2_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8000 entries, 987231 to 534950
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   fraud_bool                        8000 non-null   int64  
 1   income                            8000 non-null   float64
 2   name_email_similarity             8000 non-null   float64
 3   customer_age                      8000 non-null   int64  
 4   zip_count_4w                      8000 non-null   int64  
 5   bank_branch_count_8w              8000 non-null   int64  
 6   date_of_birth_distinct_emails_4w  8000 non-null   int64  
 7   credit_risk_score                 8000 non-null   int64  
 8   email_is_free                     8000 non-null   int64  
 9   phone_mobile_valid                8000 non-null   int64  
 10  has_other_cards                   8000 non-null   int64  
 11  proposed_credit_limit             8000 non-null   float64
 12  fore

# Train RF

Upsample minority class

In [101]:
df_majority = v2_sample[v2_sample['fraud_bool'] == 0]
df_minority = v2_sample[v2_sample['fraud_bool'] == 1]

# Calculate the number of samples for upsampling
#n_samples = int(minority_ratio * len(df_majority) / (1 - minority_ratio)) #proportional group sizes
n_samples = len(df_majority) # equal class sizes

df_minority_upsampled = resample(df_minority, replace=True, n_samples=n_samples, random_state=42)

# Combine the upsampled minority class with the majority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])

v2_upsampled = df_upsampled

In [102]:
X = v2_upsampled.drop(columns = ['fraud_bool'], axis = 1)
Y = v2_upsampled['fraud_bool']
Y.value_counts()

,count
fraud_bool,
0,7918
1,7918


Change booleans to numeric for shap to work later

In [103]:
X = X.astype(int)
Y = Y.astype(int)
X.iloc[7]

,732057
income,0
name_email_similarity,0
customer_age,20
zip_count_4w,990
bank_branch_count_8w,9
date_of_birth_distinct_emails_4w,6
credit_risk_score,116
email_is_free,1
phone_mobile_valid,1
has_other_cards,1


Create scaled versions of the features

In [104]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled = pd.DataFrame(X_scaled, columns = X.columns)

# Name columns as '_scaled'
X_scaled = X_scaled.rename(columns={col: col + '_scaled' for col in X_scaled.columns})

# Join the DataFrames
X = X.join(X_scaled)

# Display updated dataframe
X.info()
#

<class 'pandas.core.frame.DataFrame'>
Index: 15836 entries, 987231 to 496375
Data columns (total 42 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   income                                   15836 non-null  int64  
 1   name_email_similarity                    15836 non-null  int64  
 2   customer_age                             15836 non-null  int64  
 3   zip_count_4w                             15836 non-null  int64  
 4   bank_branch_count_8w                     15836 non-null  int64  
 5   date_of_birth_distinct_emails_4w         15836 non-null  int64  
 6   credit_risk_score                        15836 non-null  int64  
 7   email_is_free                            15836 non-null  int64  
 8   phone_mobile_valid                       15836 non-null  int64  
 9   has_other_cards                          15836 non-null  int64  
 10  proposed_credit_limit                    1583

Train on scaled X (unnecessary for RF but good for SHAP later on)

In [ ]:
#training on a scaled X unnecessary for random forest but good for shap later
X_all_train, X_all_test, Y_train, Y_test = train_test_split(X, Y,
                                                    #test_size = 0.3,
                                                    test_size = 0.7, shuffle = True, stratify = Y)

X_train = X.drop([], axis=1)
X_test = X.drop([], axis=1)


model = RandomForestClassifier()
model.fit(X_train, Y_train)

#predictions
Yhat = model.predict(X_test)
len(Yhat)